In [ ]:
import xarray
import pvxarray
from pvxarray.vtk_source import PyVistaXarraySource


In [ ]:
# Temporary manual mode override; waiting for pyvista release including changes from https://github.com/pyvista/pyvista/pull/5798
import pyvista as pv 

pv.global_theme.trame.server_proxy_enabled = False
pv.global_theme.trame.jupyter_extension_enabled = True

In [ ]:
url = 'https://ncsa.osn.xsede.org/Pangeo/pangeo-forge/noaa-coastwatch-geopolar-sst-feedstock/noaa-coastwatch-geopolar-sst.zarr'
dataset = xarray.open_dataset(url, engine="zarr", chunks={})

algorithm = PyVistaXarraySource()
algorithm.data_array = dataset["analysed_sst"]
algorithm.resolution = 0.01
algorithm.x = "lat"
algorithm.y = "lon"
algorithm.time = "time"
algorithm.time_index = 0


In [ ]:
mesh = algorithm.apply()
mesh.plot()
